In [40]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.llms import Ollama
from langchain.chains import LLMChain

def get_model():
    return Ollama(
        model="mistral", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()])
    )

model = get_model()


In [76]:
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory


template = """You are a chatbot having a conversation with a human. Keep responses as small as possible.
Use this context as appropriate: {context}

{chat_history}
Human: {human_input}
Chatbot: """

prompt = ChatPromptTemplate.from_template(template)
memory = ConversationBufferMemory(memory_key="chat_history")

In [53]:
from langchain.chains import ConversationChain
chain = LLMChain(
    llm=model,
    prompt=prompt,
    verbose=True,
    memory=memory)

## Adding Docs

In [91]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.gpt4all import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

raw_data = TextLoader("./data.txt").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
data = text_splitter.split_documents(raw_data)
vectorstore = Chroma.from_documents(documents=data, embedding=GPT4AllEmbeddings())

retriever = vectorstore.as_retriever()


bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [124]:
from langchain.prompts.prompt import PromptTemplate

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [125]:
template = """Answer the question based only on the following context, and the associated history:
History: 
{chat_history}

Context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [79]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

In [98]:
from typing import List, Tuple


def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "Human: " + dialogue_turn[0]
        ai = "Assistant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer


In [132]:
from operator import itemgetter
from langchain.schema import format_document
from typing import List, Tuple
from langchain.schema.runnable import RunnableMap, RunnablePassthrough, RunnableLambda

memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | model
    | StrOutputParser(),
    "chat_history": lambda x: _format_chat_history(x["chat_history"])
}

# Now we retrieve the documents
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
    "chat_history": lambda x: x["chat_history"],
    
}
# Now we construct the inputs for the final prompt
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
    "chat_history": lambda x: x["chat_history"],
}
# And finally, we do the part that returns the answers
answer = {
    "answer": final_inputs | ANSWER_PROMPT | model,
    # "docs": itemgetter("docs"),
    "history": itemgetter("chat_history")
}


# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [137]:
def ask_question(question):
    inputs = {"question": question}
    result = final_chain.invoke(inputs)
    # For now you need to save it yourself
    memory.save_context(inputs, {"answer": result["answer"]})
    return result
    

In [139]:
ask_question("What is my name?")

TypeError: 'HumanMessage' object is not subscriptable